<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Alberto Vavassori</b> (alberto.vavassori@polimi.it), <b>Emanuele Capizzi</b> (emanuele.capizzi@polimi.it) - DICA - Politecnico di Milano - GIS GEOLab <br>
Developed within the LCZ-ODC project, funded by the Italian Space Agency (agreement n. 2022-30-HH.0).
</div>

# LCZ classification accuracy assessment

<a id='TOC_TOP'></a>
Notebook structure:  <br>

[Part 1: Classification accuracy on testing samples](#sec1.0)

 1. [Import testing samples](#sec1)  
 2. [Rasterize testing samples](#sec2)
 3. [Import the classified image to be assessed](#sec3)
 4. [Assess classification accuracy on testing samples](#sec4)
 
[Part 2: Inter-comparison with LCZ Generator product](#sec2.0)

 5. [Accuracy of LCZ Generator product](#sec5)
 6. [Extraction of samples for inter-comparison](#sec6)
 7. [Computation of confusion matrix and consistency metrics](#sec7)
 
<hr>

This Notebook is meant to verify the quality of the classification using testing samples. These samples consist of an external dataset that was not used within the classification step. The testing samples are always defined by the user and can be imported into this Notebook.

### Import libraries

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
Note: the Notebook relies on the <a href='https://gdal.org/' target='_blank'><em>gdal</em></a> Python library; make sure you have it installed in your environment.
</div>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import glob
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.windows import Window
from rasterio.enums import Resampling
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
from osgeo import gdal, ogr, gdalconst, gdal_array, osr
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from shapely.geometry import box

In [ ]:
# Import functions and set auto-reload
from functions import *
%load_ext autoreload
%autoreload 2

<div class="alert alert-success">

# <a id='sec1.0'></a> Part 1: Classification accuracy on testing samples
<a id='sec2'></a>
[Back to top](#TOC_TOP)

</div>

## 1. <a id='sec1'></a> Import testing samples
[Back to top](#TOC_TOP)

First, the following dictionary creates the correspondance between the date of acquisition of Sentinel-2 (key) and of PRISMA (value):

In [ ]:
prisma_s2_dates = {'2023-02-10': '2023-02-09',
                   '2023-03-22': '2023-03-22',
                   '2023-04-26': '2023-04-08',
                   '2023-06-25': '2023-06-17',
                   '2023-07-10': '2023-07-10',
                   '2023-08-19': '2023-08-08'
}

First, select the Sentinel-2 image date:

In [ ]:
date_s2_w = widgets.Dropdown(
    options=['2023-02-10', '2023-03-22', '2023-04-26', '2023-06-25', '2023-07-10', '2023-08-19'],
    value='2023-02-10',
    description='Sentinel-2 date:',
    disabled=False,
    layout={'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_s2_w

In [ ]:
sel_s2_date = date_s2_w.value
selected_s2_image = 'S2_outputs/S2_'+ sel_s2_date.replace('-', '') + '_20m_clip.tif'
selected_prisma_image = 'PRISMA_outputs/coregistered/PR_VNIR_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_30m.tif'
print(f"The selected date is --> Sentinel-2: {sel_s2_date}.")

Set the legend that will be used henceforth for the plots:

In [ ]:
legend = {
    2: ['Compact mid-rise', '#D10000'],
    3: ['Compact low-rise', '#CD0000'],
    5: ['Open mid-rise', '#FF6600'],
    6: ['Open low-rise', '#FF9955'],
    8: ['Large low-rise', '#BCBCBC'],
    101: ['Dense trees', '#006A00'],
    102: ['Scattered trees', '#00AA00'],
    104: ['Low plants', '#B9DB79'],
    105: ['Bare rock or paved', '#545454'],
    106: ['Bare soil or sand', '#FBF7AF'],
    107: ['Water', '#6A6AFF']
}

The following function displays the testing samples on a map.

In [ ]:
testing_folder = './layers/testing_samples/testing_set_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '.gpkg'
cmm_folder = './layers/CMM.gpkg'

In [ ]:
testing, m, shapes = plot_training_samples(testing_folder, cmm_folder, legend)

In [ ]:
m

The following function imports the testing samples and computes the area of each LCZ class. The function outputs a plot with the total area of each LCZ class as well as the path to the vector layer that will be used in the following.

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> 

**Note**:
Testing samples must be stored in `.gpkg` format as vector multi-polygons, and the file must contain a column `LCZ` with integer values corresponding to the LCZ class, as reported in the dictionary `legend`.

</div>
<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> Testing samples are time-dependent, especially for the natural classes; they must be updated if there are changes (e.g. in the land cover).  The following function imports the training samples specific to the selected PRISMA acquisition date.
</div>

In [ ]:
vector_LCZ_path = testing_area(prisma_s2_dates[sel_s2_date], legend)

## 2. <a id='sec2'></a>Rasterize testing samples
[Back to top](#TOC_TOP)

</div>
<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> If testing samples are already rasterized, skip this section and go to the next one.
</div>

It is necessary to convert the testing set (provided in vector format as Geopackage) in raster format using the following functions.

In [ ]:
raster_reference = 'S2_outputs/S2_'+ sel_s2_date.replace('-', '') +'_20m_clip.tif'
output = './layers/testing_samples/testing_set_'+ prisma_s2_dates[sel_s2_date].replace('-', '') + '_20m.tif'
attribute = 'LCZ'
projection = 32632

In [ ]:
rasterized_result = rasterize_training('S2_outputs/S2_' + sel_s2_date.replace('-', '') + '_20m.tif', vector_LCZ_path, output, attribute, projection)

In [ ]:
#plot_raster_training(output, legend)

In [ ]:
bbox = prisma_bbox(selected_prisma_image, prisma_s2_dates[sel_s2_date])

In [ ]:
mask_prisma = mask_prisma_image(selected_prisma_image)

In [ ]:
study_area = gpd.read_file(bbox)

In [ ]:
roi_ds = './layers/testing_samples/testing_set_'+ prisma_s2_dates[sel_s2_date].replace('-', '')+'_20m.tif'
roi_new_path = './layers/testing_samples/testing_set_'+ prisma_s2_dates[sel_s2_date].replace('-', '')+'_20m.tif'
clip_image_study_area(roi_ds, roi_new_path, study_area)

## 3. <a id='sec3'></a>Import the classified image to be assessed
[Back to top](#TOC_TOP)

Select the method used for classification to chose the image that want to be tested:

In [ ]:
classification_method_w = widgets.RadioButtons(
    options=['RandomForest', 'XGBoost', 'AdaBoost', 'GradientBoost'],
    value='RandomForest',
    layout={'width': 'max-content'},
    description='Classifier:',
    disabled=False
)
classification_method_w

In [ ]:
classification_method = classification_method_w.value
print(f'Selected classification method: {classification_method}')

In [ ]:
print('Selected image: classified_images/classified_' + classification_method + '_' + sel_s2_date.replace('-', '') + '_medianfilter_20m_align_S2_masked.tif')
classified_image = rasterio.open('classified_images/comparison_PRISMA_S2/classified_' + classification_method + '_' + sel_s2_date.replace('-', '') + '_medianfilter_20m_align_S2_masked.tif')
classified_image = classified_image.read()
print(f"Selected image shape: {classified_image.shape}")

In [ ]:
plot_classified_image(sel_s2_date, classified_image, classification_method, legend)

## 4. <a id='sec4'></a>Assess classification accuracy on testing samples
[Back to top](#TOC_TOP)

In this section, some accuracy metrics are computed on the testing samples, specifically:
* accuracy: overall accuracy of the model, i.e. the fraction of the total samples that were correctly classified

$$ \frac{TP+TN}{TP+TN+FP+FN} $$

* precision: fraction of predictions as a positive class were actually positive

$$ \frac{TP}{TP+FP} $$

* recall: fraction of all positive samples that are correctly predicted as positive

$$ \frac{TP}{TP+FN} $$

* f1-score: combination of precision and recall; mathematically it is the harmonic mean of precision and recall

$$\frac{2(precision*recall)}{precision+recall}$$

* support: number of occurrences of each class in the testing sample.

In [ ]:
accuracy, confusion, report, report_df = print_accuracy_s2(classification_method, sel_s2_date, prisma_s2_dates, legend)

In [ ]:
confusion

In [ ]:
report_df

<div class="alert alert-success">

# <a id='sec2.0'></a> Part 2: Inter-comparison with LCZ Generator product
<a id='sec2'></a>
[Back to top](#TOC_TOP)

</div>

## 5. <a id='sec5'></a> Accuracy of LCZ Generator product
[Back to top](#TOC_TOP)

The accuracy of the LCZ Generator product is evaluated internally, and it is provided as output. Here we read the confusion matrix from the csv file, and we compute accuracy metrics.

In [ ]:
#directory = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/data/*cm_average_formatted.csv'
#confusion_matrix_path = glob.glob(directory)[0]

In [ ]:
#report = lcz_generator_accuracy(confusion_matrix_path, legend)

In [ ]:
#report

## 6. <a id='sec6'></a>Extraction of samples for inter-comparison with LCZ Generator product and with the PRISMA-derived map
[Back to top](#TOC_TOP)

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
This section needs some pre-processing that can be performed entirely in QGIS. Specifically:
</div>

* The two maps have to be resampled to 10m resolution in QGIS, given that they have different spatial resolution (100m and 30m).
* The LCZ Generator map must be aligned to the PRISMA map.
* The two classified maps must finally be post-processed with a median filter of 9 pixels (the size is coherent with the 3 pixel window size that is used for post-processing the PRISMA map in the LCZ-ODC project workflow).
* Finally, the LCZ Generator map must be reclassified so that the LCZ classes are encoded as in the PRISMA map.

Once the above pre-processing is done, clip the LCZ Generator map to the extent of the PRISMA map (this is crucial because the PRISMA map is rotated with respect to the other one due to the acquisition mode of the satellite).

In [ ]:
# Paths to your input rasters (pre-processed in QGIS)
lcz_generator_map_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass.tif'
s2_map_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9.tif'

In [ ]:
with rasterio.open(lcz_generator_map_path) as src1:
    lcz_generator_map = src1.read()
    lcz_generator_map_profile = src1.profile

In [ ]:
with rasterio.open(s2_map_path) as src2:
    s2_map = src2.read()
    s2_map_profile = src2.profile

In [ ]:
s2_map[s2_map < 0] = np.nan
nan_indices = np.isnan(s2_map)
lcz_generator_map[nan_indices] = np.nan

In [ ]:
with rasterio.open('./layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip.tif', 'w', **lcz_generator_map_profile) as dst:
    dst.write(lcz_generator_map)

In [ ]:
with rasterio.open('./layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass.tif', 'w', **lcz_generator_map_profile) as dst:
    dst.write(s2_map)

The inter-comparison is hereafter carried out following 3 sampling schemas.

<u>Sampling schema: random sampling with fixed pixel number<u>

This first sampling schema considers a fixed number of pixels, calculated using the Cochran's formula for large populations.
Considering a precision level of +/-3%, a confidence level of 98%, and an estimated proportion of 0.5, the appropriate sample size is about 1500.
In the random sampling, the same number of pixels is extracted out of each class, equal to 1500/n (n being the number of classes). In this case, n = 11.

In [ ]:
raster1_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip.tif'
raster2_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass.tif'
raster3_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass.tif'

In [ ]:
output_raster1_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip_pixels.tif'
output_raster2_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass_pixels.tif'
output_raster3_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_pixels.tif'

In [ ]:
pixels_number = 1500

In [ ]:
random_sampling(raster1_path, raster2_path, raster3_path, output_raster1_path, output_raster2_path, output_raster3_path, pixels_number, legend.keys())

<u>Sampling schema: stratified random sampling with fixed pixel number<u>

A second sampling schema considers a fixed number of total pixels, calculated using the Cochran's formula for large populations, and subdivides it to each class (or stratum) according to its size (number of pixels). Considering a precision level of +/-3%, a confidence level of 98%, and an estimated proportion of 0.5, the appropriate sample size is about 1500. In the stratified random sampling, the number of pixels to be extracted per class is computed as:

$$ n_h = \frac{N_h}{N} n $$

where $n_h$ is the sample size of the h-th class, $N_h$ is the population size of the h-th class, $N$ is the size of the entire population (number of pixels in the raster, excluding nans) and $n$ is the size of the entire sample (1500).

In [ ]:
raster1_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip.tif'
raster2_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass.tif'
raster3_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass.tif'

In [ ]:
output_raster1_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip_pixels_strat.tif'
output_raster2_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'
output_raster3_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'

In [ ]:
# n: size of the entire sample
n = 1500
pixels_number = n

In [ ]:
stratified_sampling(raster1_path, raster2_path, raster3_path, output_raster1_path, output_raster2_path, output_raster3_path, pixels_number, legend.keys())

<u>Sampling schema: take all pixels of the maps<u>

A third possibility consists in using all the pixels of both rasters for the comparison. If this is the case, just put as input in the below function to outputs of the first pre-processing, namely:

In [ ]:
# raster1_path = './layers/lcz_generator/comparison_20230209/lcz_generator_map_20230209_10m_align_medianfilter9_reclass_clip.tif'
# raster2_path = './layers/lcz_generator/comparison_20230209/prisma_20230209_10m_align_medianfilter9_reclass.tif'

## 7. <a id='sec7'></a>Computation of confusion matrix and consistency metrics
[Back to top](#TOC_TOP)

**PRISMA vs Sentinel-2**

In [ ]:
ref_raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'
raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'

In [ ]:
accuracy, confusion, report, report_df = inter_comparison(raster_path, ref_raster_path, legend)

In [ ]:
confusion

In [ ]:
report_df

**PRISMA vs LCZ Generator**

In [ ]:
ref_raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip_pixels_strat.tif'
raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/prisma_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'

In [ ]:
accuracy, confusion, report, report_df = inter_comparison(raster_path, ref_raster_path, legend)

In [ ]:
confusion

In [ ]:
report_df

**Sentinel-2 vs LCZ Generator**

In [ ]:
ref_raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/lcz_generator_map_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '_10m_align_medianfilter9_reclass_clip_pixels_strat.tif'
raster_path = './layers/lcz_generator/comparison_' + prisma_s2_dates[sel_s2_date].replace('-', '') + '/s2_' + sel_s2_date.replace('-', '') + '_10m_align_medianfilter9_reclass_pixels_strat.tif'

In [ ]:
accuracy, confusion, report, report_df = inter_comparison(raster_path, ref_raster_path, legend)

In [ ]:
confusion

In [ ]:
report_df

In [ ]:
classi = ["Compact Mid-rise","Compact Low-rise","Open Mid-rise","Open Low-rise","Large Low-rise",
          "Dense trees","Scattered trees","Low plants","Bare rock or paved","Bare soil or sand","Water"]

accur_s2_10feb = np.array([[2720,   50,  238,    2,   12,    0,    0,    0,    0,    0,    0],
       [ 292, 1502,  210,  339,   53,    0,    0,    0,    0,    0,    0],
       [ 384,   90, 1960,  155,    7,    0,    0,    0,    0,    0,    0],
       [   4,  393,  215, 1887,   10,    0,   44,    0,   50,    0,    1],
       [   0,   54,   18,   26, 2503,    0,    0,    0,   26,    1,    0],
       [   0,    0,   10,   11,    0, 2217,  369,    0,    0,    0,    0],
       [   0,    0,    2,  121,    2,  250, 2307,   17,   62,    6,    7],
       [   0,    0,    0,    1,    0,    0,   97, 2618,    0,    0,    0],
       [   0,    0,    0,   31,    2,    0,   10,   11,  625,    3,    0],
       [   0,    0,    0,    0,   13,    0,    0,    0,    0, 2789,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1450]])
accur_s2_10feb = accur_s2_10feb / np.sum(accur_s2_10feb)*100

accur_s2_25jun = np.array([[2570,   76,  356,    0,   20,    0,    0,    0,    0,    0,    0],
       [ 308, 1542,  194,  321,   30,    0,    0,    0,    0,    0,    1],
       [ 380,   89, 1981,  145,    1,    0,    0,    0,    0,    0,    0],
       [  10,  393,  197, 1909,   19,    0,   59,    2,   13,    2,    0],
       [  18,  103,   19,   65, 2193,    0,    0,    0,  230,    0,    0],
       [   0,    0,    6,   19,    0, 1354,  528,    6,    0,    0,    0],
       [   0,    0,   15,   98,    2,  328, 2246,   18,   39,   23,    5],
       [   0,    0,    0,    0,    0,    1,   68, 3637,    0,  292,    4],
       [   0,    0,    0,   25,    2,    0,    1,   14,  640,    0,    0],
       [   0,    0,    0,    0,   12,    0,   52,  413,    0, 2463,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1450]])
accur_s2_25jun = accur_s2_25jun / np.sum(accur_s2_25jun)*100

accur_s2_19aug = np.array([[2625,   55,  315,    3,   24,    0,    0,    0,    0,    0,    0],
       [ 328, 1273,  225,  167,   40,    0,    0,    0,    0,    0,    0],
       [ 384,   83, 2011,  112,    6,    0,    0,    0,    0,    0,    0],
       [  10,  373,  269, 1457,    8,    0,   69,    1,   16,    3,    0],
       [  11,   17,    7,   29, 2507,    0,    0,    0,   56,    1,    0],
       [   0,    0,   14,   20,    0,  699,  508,    5,    0,    0,    0],
       [   0,    0,   19,   94,    7,  271, 2408,   51,   49,    2,   15],
       [   0,    0,    0,    0,    0,    0,   61, 4943,    0,  323,    0],
       [   0,    0,    0,   13,   17,    0,    0,   43, 1260,   10,    0],
       [   0,    0,    0,    0,   28,    0,   27,  677,    0, 1781,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1146]])
accur_s2_19aug = accur_s2_19aug / np.sum(accur_s2_19aug)*100

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = accur_s2_10feb)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(accur_s2_10feb, ax = ax1, cmap=cmap, vmin = 0, vmax = 14,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("Testing samples", fontsize = 15)
ax1.set_xlabel("Sentinel-2 map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Confusion matrix computed from testing samples\nSentinel-2', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/accuracy_s2_feb_test_samples.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = accur_s2_25jun)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(accur_s2_25jun, ax = ax1, cmap=cmap, vmin = 0, vmax = 14,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("Testing samples", fontsize = 15)
ax1.set_xlabel("Sentinel-2 map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Confusion matrix computed from testing samples\nSentinel-2', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/accuracy_s2_jun_test_samples.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = accur_s2_19aug)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(accur_s2_19aug, ax = ax1, cmap=cmap, vmin = 0, vmax = 14,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("Testing samples", fontsize = 15)
ax1.set_xlabel("Sentinel-2 map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Confusion matrix computed from testing samples\nSentinel-2', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/accuracy_s2_aug_test_samples.png', dpi = 300)
plt.show()

In [ ]:
s2_prisma_feb = np.array([[ 40,   1,   3,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,  33,  10,  10,   8,   0,   0,   0,   0,   0,   0],
       [  4,   0, 100,  23,   1,   0,   0,   0,   0,   0,   0],
       [  0,   1,   8, 175,  11,   1,   8,   4,   4,   0,   0],
       [  0,   1,   5,  11, 105,   0,   2,   0,   2,   0,   0],
       [  0,   0,   0,   1,   0,  20,  12,   0,   0,   1,   0],
       [  0,   0,   0,  11,   1,   9, 111,  38,   2,   2,   0],
       [  0,   0,   0,   0,   0,   0,  23, 267,   3,  16,   0],
       [  0,   0,   0,   9,   6,   0,  11,   9,  49,   2,   0],
       [  0,   0,   0,   0,   2,   0,   4,  14,   4, 278,   1],
       [  0,   0,   0,   0,   0,   0,   0,   1,   1,   3,  10]])
s2_prisma_feb = s2_prisma_feb / np.sum(s2_prisma_feb)*100
s2_lcz_gen_feb = np.array([[ 37,   8,   1,   0,   1,   0,   0,   0,   0,   0,   0],
       [  5,  21,   2,   7,   1,   0,   0,   0,   0,   0,   0],
       [ 14,  10,  57,  37,   5,   0,   2,   0,   1,   0,   0],
       [  2,  15,  24, 159,  14,   1,   7,   6,   6,   6,   0],
       [  0,  10,   9,  18,  86,   0,   3,   1,   5,   2,   0],
       [  0,   0,   0,   2,   0,  20,   3,   3,   2,   0,   0],
       [  0,   0,   2,  19,   2,   8,  82,  38,  10,   9,   1],
       [  0,   0,   0,   7,   0,   3,  16, 206,   6,  95,   0],
       [  0,   3,   5,  12,   7,   2,   6,   3,  16,  11,   0],
       [  0,   0,   0,   5,   2,   0,   8,  69,   5, 213,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   1,   0,  10]])
s2_lcz_gen_feb = s2_lcz_gen_feb / np.sum(s2_lcz_gen_feb)*100
prisma_lcz_gen_feb = np.array([[ 37,   6,   1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,  31,   4,  14,   7,   0,   0,   0,   1,   0,   0],
       [ 13,  11,  57,  40,   4,   0,   2,   1,   0,   0,   0],
       [  0,   8,  23, 147,   7,   1,   6,  10,   5,   5,   0],
       [  1,   7,   6,  17,  90,   0,   0,   0,   4,   1,   0],
       [  0,   0,   0,   3,   0,  17,  10,   3,   1,   0,   0],
       [  0,   0,   1,  18,   0,  11,  74,  52,   5,  12,   1],
       [  0,   0,   1,   8,   0,   3,  18, 191,   2,  86,   0],
       [  0,   4,   7,  13,   8,   0,   7,   8,  27,  12,   0],
       [  0,   0,   0,   6,   2,   1,   8,  60,   6, 219,   1],
       [  0,   0,   0,   0,   0,   1,   2,   1,   1,   1,   9]])
prisma_lcz_gen_feb = prisma_lcz_gen_feb / np.sum(prisma_lcz_gen_feb)*100
############################################################################
s2_prisma_jun = np.array([[ 39,   0,   6,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,  32,   5,   5,   4,   0,   0,   0,   0,   0,   0],
       [  3,   3,  74,  23,   0,   0,   0,   0,   0,   0,   0],
       [  0,   6,  19, 207,  10,   1,   8,   2,   0,   0,   1],
       [  0,   1,   5,  31, 109,   0,   3,   1,   5,   0,   0],
       [  0,   0,   0,   0,   0,  12,  17,   1,   0,   0,   0],
       [  0,   0,   0,  13,   0,   4, 112,  21,   0,  12,   1],
       [  0,   0,   0,   3,   2,   1,  26, 359,   1,  13,   2],
       [  0,   0,   0,  10,  10,   0,  13,  12,  39,   6,   0],
       [  0,   0,   0,   0,   2,   0,   4,  65,   2, 117,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   9]])
s2_prisma_jun = s2_prisma_jun / np.sum(s2_prisma_jun)*100
s2_lcz_gen_jun = np.array([[ 36,   6,   1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  5,  26,   1,   9,   0,   0,   0,   0,   1,   0,   0],
       [ 11,   9,  56,  28,   2,   0,   1,   0,   2,   0,   0],
       [  3,  14,  28, 186,  19,   0,  16,   9,   8,   9,   0],
       [  5,   9,  10,  14,  88,   0,   3,   0,   4,   4,   0],
       [  0,   0,   1,   1,   0,  13,   2,   0,   0,   1,   0],
       [  0,   1,   3,  29,   4,  15,  85,  28,   4,  14,   0],
       [  0,   0,   2,  12,   0,   5,  43, 274,   7, 118,   0],
       [  0,   0,   3,   7,   6,   0,   1,   4,  21,   4,   1],
       [  0,   0,   0,   3,   0,   0,   4,  71,   3,  68,   0],
       [  0,   0,   0,   0,   0,   0,   1,   2,   0,   1,   9]])
s2_lcz_gen_jun = s2_lcz_gen_jun / np.sum(s2_lcz_gen_jun)*100
prisma_lcz_gen_jun = np.array([[ 34,   7,   2,   1,   1,   0,   0,   0,   0,   0,   0],
       [  8,  26,   3,   7,   2,   0,   0,   0,   1,   0,   0],
       [ 10,  11,  58,  20,   2,   0,   1,   0,   1,   0,   0],
       [  1,  11,  18, 179,   7,   0,  15,   9,   5,   9,   0],
       [  5,   9,  12,  19,  96,   0,   3,   0,   7,   4,   0],
       [  0,   0,   1,   5,   0,  15,   8,   1,   0,   0,   0],
       [  0,   1,   2,  23,   2,  14,  77,  20,   5,  19,   0],
       [  0,   0,   1,  10,   0,   3,  35, 253,   7,  97,   1],
       [  2,   0,   7,  20,   7,   0,   5,   9,  23,  16,   1],
       [  0,   0,   1,   5,   2,   1,  12,  94,   1,  74,   0],
       [  0,   0,   0,   0,   0,   0,   0,   2,   0,   0,   8]])
prisma_lcz_gen_jun = prisma_lcz_gen_jun / np.sum(prisma_lcz_gen_jun)*100
############################################################################
s2_prisma_aug = np.array([[ 34,   4,   5,   0,   1,   0,   0,   0,   0,   0,   0],
       [  3,  36,   7,   9,   2,   0,   0,   0,   0,   0,   0],
       [  1,   1, 126,  27,   4,   0,   1,   0,   0,   0,   0],
       [  0,   5,  17, 144,  10,   0,   8,   5,   7,   0,   0],
       [  0,   3,   1,  16, 112,   0,   2,   0,   5,   3,   0],
       [  0,   0,   0,   2,   1,   6,   9,   0,   0,   0,   0],
       [  0,   0,   1,  14,   2,   2,  80,   5,   2,   0,   0],
       [  0,   0,   0,  10,   1,   2,  60, 465,   9,  30,   0],
       [  0,   0,   0,   9,  14,   0,   8,   7,  50,   4,   0],
       [  0,   0,   0,   0,   3,   0,   5,  37,   4,  35,   0],
       [  0,   0,   0,   1,   0,   0,   5,   1,   0,   1,  10]])
s2_prisma_aug = s2_prisma_aug / np.sum(s2_prisma_aug)*100
s2_lcz_gen_aug = np.array([[ 34,   2,   1,   0,   1,   0,   0,   0,   0,   0,   0],
       [  9,  21,   4,  14,   1,   0,   0,   0,   0,   0,   0],
       [ 15,  13,  74,  42,   4,   4,   3,   1,   1,   0,   0],
       [  2,  19,  40,  92,  12,   3,  39,  15,   4,   6,   0],
       [  6,  24,   9,  23,  72,   0,   5,   7,   3,   1,   0],
       [  0,   0,   0,   0,   0,   8,   2,   0,   0,   0,   0],
       [  0,   1,   5,   3,   4,  12, 105,  39,   1,   8,   0],
       [  0,   2,   3,   4,   1,   1,  63, 405,   2,  39,   0],
       [  4,   5,   7,  13,   8,   0,   7,  21,   7,   5,   0],
       [  0,   1,   1,   3,   2,   0,  13,  37,   0,  16,   0],
       [  0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   9]])
s2_lcz_gen_aug = s2_lcz_gen_aug / np.sum(s2_lcz_gen_aug)*100
prisma_lcz_gen_aug = np.array([[ 36,   3,   4,   0,   1,   0,   0,   0,   0,   0,   0],
       [  9,  23,  11,  13,   1,   0,   0,   0,   0,   0,   0],
       [ 12,  17,  78,  33,  10,   2,   5,   1,   1,   1,   0],
       [  3,  10,  28,  95,   7,   3,  31,  15,   2,   2,   0],
       [  5,  25,   5,  24,  70,   0,   3,   5,   2,   3,   0],
       [  0,   0,   1,   0,   0,   8,   8,   0,   0,   1,   0],
       [  0,   0,   4,   1,   1,   9,  69,  18,   1,   3,   0],
       [  0,   2,   3,  10,   2,   6,  95, 410,   2,  47,   0],
       [  5,   8,   8,  16,   7,   0,   8,  23,  10,   7,   0],
       [  0,   0,   2,   2,   6,   0,  13,  50,   0,  11,   0],
       [  0,   0,   0,   0,   0,   0,   5,   4,   0,   0,   9]])
prisma_lcz_gen_aug = prisma_lcz_gen_aug / np.sum(prisma_lcz_gen_aug)*100

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = s2_prisma_feb)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(s2_prisma_feb, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("PRISMA map", fontsize = 15)
ax1.set_xlabel("Sentinel-2 map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nPRISMA vs Sentinel-2', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_s2_prisma_feb.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = s2_lcz_gen_feb)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(s2_lcz_gen_feb, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("Sentinel-2 map", fontsize = 15)
ax1.set_xlabel("LCZ Generator map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nSentinel-2 vs LCZ Generator', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_s2_lcz_gen_feb.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = prisma_lcz_gen_feb)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(prisma_lcz_gen_feb, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("PRISMA map", fontsize = 15)
ax1.set_xlabel("LCZ Generator map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nPRISMA vs LCZ Generator', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_prisma_lcz_gen_feb.png', dpi = 300)
plt.show()

In [ ]:
## JUNE ##########################################

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = s2_prisma_jun)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(s2_prisma_jun, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("PRISMA map", fontsize = 15)
ax1.set_xlabel("Sentinel-2 map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nPRISMA vs Sentinel-2', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_s2_prisma_jun.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = s2_lcz_gen_jun)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(s2_lcz_gen_jun, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("Sentinel-2 map", fontsize = 15)
ax1.set_xlabel("LCZ Generator map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nSentinel-2 vs LCZ Generator', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_s2_lcz_gen_jun.png', dpi = 300)
plt.show()

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Create a DataFrame from the matrix
accuracy_prisma = pd.DataFrame(data = prisma_lcz_gen_jun)

# Generate a mask for the upper triangle

# Set up the matplotlib figure
f, ax1 = plt.subplots(figsize=(10, 10))
plt.tight_layout()

#cbar_ax = f.add_axes([0.25, 0.01, 0.5, 0.02])  # Adjust the position and size as needed

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
cmap = 'Greens'
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(prisma_lcz_gen_jun, ax = ax1, cmap=cmap, vmin=0, vmax=25,
            square=True, linewidths=.5, cbar_kws={"shrink": .5, "use_gridspec": True}, #, "orientation": "horizontal"
            annot=True, fmt=".1f", annot_kws={"fontsize": 13, "color": 'black'}) #, cbar_ax = cbar_ax

# Set x and y ticks
ax1.set_xticks(np.arange(len(classi)) + 0.5)
ax1.set_yticks(np.arange(len(classi)) + 0.5)
ax1.set_ylabel("PRISMA map", fontsize = 15)
ax1.set_xlabel("LCZ Generator map", fontsize = 15)

# Set x and y tick labels
ax1.set_xticklabels(classi, rotation = 90, ha = "right", fontsize = 15)
ax1.set_yticklabels(classi, rotation = 0, va = "center", fontsize = 15)
ax1.set_title('Inter-comparison\nPRISMA vs LCZ Generator', fontweight = 'bold', fontsize = 15)

plt.subplots_adjust(left = 0.30, right = 0.9, bottom = 0.2, top = 0.9)
plt.savefig('classified_images/IC_prisma_lcz_gen_jun.png', dpi = 300)
plt.show()